## Building GPT from Scratch

Building a gpt model without tokenization. The inputs are the characters 
1. Build biagram in a pytorch form, with a Module function 
 

In [115]:
import torch

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

Connecting to raw.githubusercontent.com (185.199.110.133:443)
saving to 'input.txt'
input.txt            100% |********************************| 1089k  0:00:00 ETA
'input.txt' saved


In [116]:
with open('input.txt','r',encoding='utf-8') as f:
    text = f.read()
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [117]:
len(text)

1115394

In [118]:
chars = sorted(list(set(text)))
print (chars)
stoi = {i:k for k,i in enumerate(chars)}
itos = {i:k for k,i in stoi.items()}


['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [119]:
encode = lambda s: [stoi[i] for i in s ]     # takes a string and output a list of integers
encode('hola')
decode = lambda li: ''.join(itos[i] for i in li )
print (encode('Hola'))
decode(encode('Hola'))

[20, 53, 50, 39]


'Hola'

In [120]:
data = torch.tensor(encode(text), dtype=torch.long)     # We converted the data in integers 
data[:100]      
# Many specific layers, such as nn.Embedding and loss functions like CrossEntropyLoss and coded to accept only longTensor(int64) for indices  

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [121]:
# Divide the set in two 
n = int(len(data)*0.9)
train_data = data[:n]
val_data = data[n:]

In [122]:
blockSize = 8
x = train_data[:blockSize]
y = train_data[1:blockSize+1]
for t in range(blockSize):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [123]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def getBacth(split):
    data = train_data if split == 'train' else val_data
    # We need to select x batches from the data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = getBacth('train')        # Size is [B,T]         B = Batch size   T=Blocksize or time that is how far in the past can our model analyze data
for t in range(blockSize):
    print(f"when input is {xb[0][:t+1]} the target: {yb[0][t]}")

when input is tensor([24]) the target: 43
when input is tensor([24, 43]) the target: 58
when input is tensor([24, 43, 58]) the target: 5
when input is tensor([24, 43, 58,  5]) the target: 57
when input is tensor([24, 43, 58,  5, 57]) the target: 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target: 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target: 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target: 39


In [124]:
xb # Is our first input 

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [125]:
import torch.nn as nn
from torch.nn import functional as F 
torch.manual_seed(1337)

vocab_size = len(chars)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)        # Create a vocalSize x Vocalsize matrix
    
    def forward(self, idx, target = None):
        logist = self.token_embedding_table(idx) # B,T,C
        # Target = B, T 
        # PyTorch expects the "Classes/Vocabulary" dimension (C) to be the second dimension.
        # We need to flatten it 
        if target is None: 
            loss = None
        else : 
            B, T, C  = logist.shape
            logist = logist.view(B*T,C)
            target = target.view(B*T)
            loss = F.cross_entropy(logist,target)   
        # We return the logist as (B*T,C)
        return logist, loss
    
    def generate(self,idx, max_new_tokens):
        # WE GENERATE a new characters for each batch so we create 4 for each loop
        # Idx It holds the history of the conversation so far.
        # The model predicts what comes after the last l (predicts 'o'). Fo
        for _ in range(max_new_tokens):
            logits,loss = self(idx)     # self is a embbeding table
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C) takes the last character of each batch
            probs = F.softmax(logits, dim=-1) # (B, C) In PyTorch, dim=-1 means "apply the operation along the very last dimension."
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
        

m = BigramLanguageModel()

logits, loss = m.forward(xb)
logits.shape
generation = m.generate(idx = torch.zeros((4, 1), dtype=torch.long), max_new_tokens=15)
print(generation)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))    # ITS very bad (is random xd, well worst than random)

tensor([[ 0, 31, 55, 41, 53, 58, 12, 54,  8, 49,  4, 50, 18, 46, 18,  3],
        [ 0, 31, 32, 17, 32, 10, 15, 17, 30, 48, 55, 28, 63, 48, 23, 59],
        [ 0, 31,  8, 24, 28,  1, 55,  9, 31, 23, 11, 61, 61, 44, 12, 17],
        [ 0, 22, 35, 25, 60, 60, 52,  9,  2,  8, 48, 45, 15, 25, 48,  9]])

bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJt-wBpm&yiltNCjeO3:Cx&vvMYW-txjuAd IRFbTpJ$zkZelxZtTlHNzdXXUi


In [126]:
"""
OPTIMIZER EXPLANATION: AdamW vs. SGD

1. Stochastic Gradient Descent (SGD):
   - Like a hiker in the fog taking steps based only on the current slope.
   - Has a fixed learning rate for all parameters.
   - No memory of previous steps (unless momentum is added manually).
   - Can get stuck in flat areas or bounce around in narrow ravines.

2. Adam (Adaptive Moment Estimation):
   - Like a heavy ball rolling down a hill with momentum.
   - Calculates a specific learning rate for *every* individual parameter.
   - Moves faster in flat areas and slows down in steep/volatile areas.
   - Keeps track of previous gradients (momentum) to smooth out the path.

3. AdamW (Adam + Decoupled Weight Decay):
   - The standard "Adam" implementation applied weight decay (regularization) 
     incorrectly by mixing it into the gradient.
   - "AdamW" fixes this by separating weight decay from the gradient update.
   - This results in better generalization and training stability, making it 
     the industry standard for training Transformers (like GPT).
"""
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [127]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = getBacth('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.4769444465637207


In [131]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


As, o IUSt my w,

Whank'the
THek' merer, dd
We ntem lud engitheso; cer ize helour
Jginte the?
Thak o


In [133]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [162]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [167]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [168]:
wei = torch.tril(torch.ones(8,8))
wei.sum(1,keepdim=True).shape

torch.Size([8, 1])

In [169]:
wei = wei/ wei.sum(1,keepdim=True)  # (T,T) / (T,1) = (T,T)     multiplicated 8 times the sum for each colum
xbow2 = wei @ x     # (T, T)    @ (B, T, C) ----> (Broadcasting)
                    # (B ,T, T) @ (B, T, C) ----> (B, T, C) 
xbow2.shape

torch.Size([4, 8, 2])

In [170]:
torch.allclose(xbow, xbow2)

False

In [172]:
torch.allclose(xbow, xbow2, atol=1e-6)

True

In [ ]:
# Is a floating point precision xd
diff = (xbow - xbow2).abs().max()
print(f"Max difference: {diff}")

Max difference: 3.236345946788788e-08


In [ ]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = wei.softmax(dim=-1)
print(wei)
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


True

In [ ]:
tril = torch.tril(torch.ones(T,T))
wei =torch.randn(T,T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = wei.softmax(dim=-1)
print(wei)
xbow3 = wei @ x
# Depends on the value of wei, some past values are more important than other

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9273, 0.0727, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7986, 0.0195, 0.1819, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3364, 0.1693, 0.3835, 0.1107, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0382, 0.0517, 0.1715, 0.5297, 0.2089, 0.0000, 0.0000, 0.0000],
        [0.0401, 0.0419, 0.0643, 0.1913, 0.0540, 0.6084, 0.0000, 0.0000],
        [0.0980, 0.1374, 0.3441, 0.0207, 0.1610, 0.2156, 0.0233, 0.0000],
        [0.4936, 0.1135, 0.1169, 0.0446, 0.0444, 0.1178, 0.0495, 0.0198]])
